# Build your first machine learning model

In [ ]:
# Import modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Figures inline and set visualization style
%matplotlib inline
sns.set()

# Import data
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

* Below, you will drop the target 'Survived' from the training dataset and create a new DataFrame `data` that consists of training and test sets combined;
* But first, you'll store the target variable of the training data for safe keeping.

In [ ]:
# Store target variable of training data in a safe place
survived_train = df_train.Survived

# Concatenate training and test sets
data = pd.concat([df_train.drop(['Survived'], axis=1), df_test ])

* Check out your new DataFrame `data` using the `info()` method.

In [ ]:
data.info()

^ There are 2 numerical variables that have missing values: what are they?
* Impute these missing values, using the median of the of these variables where we know them:

In [ ]:
# Impute missing numerical variables
data['Age'] = data.Age.fillna(data.Age.median())
data['Fare'] = data.Fare.fillna(data.Fare.median())

# Check out info of data
data.info()

* As you want to encode your data with numbers, you'll want to change 'male' and 'female' to numbers. Use the `pandas` function `get_dummies` to do so:

In [ ]:
data = pd.get_dummies(data, columns=['Sex'], drop_first=True)
data.head()

* Select the columns `['Sex_male', 'Fare', 'Age','Pclass', 'SibSp']` from your DataFrame to build your first machine learning model:

In [ ]:
# Select columns and view head
data = data[['Sex_male', 'Fare', 'Age','Pclass', 'SibSp']]
data.head()

* Use `.info()` to check out `data`:

In [ ]:
data.info()

**Recap:**
* You've got your data in a form to build first machine learning model.

**Up next:** it's time to build your first machine learning model!


## In which you build a decision tree classifier

What is a Decision tree classsifier? It is a tree that allows you to classify data points (aka predict target variables) based on feature variables. For example,

<img src="img/decision_tree_titanic_1.png" width="400">

* You first **fit** such a model to your training data, which means deciding (based on the training data) which decisions will split at each branching point in the tree: e.g., that the first branch is on 'Male' or not and that 'Male' results in a prediction of 'Dead'. 

* Before fitting a model to your `data`, split it back into training and test sets:

In [ ]:
data_train = data.iloc[:891]
data_test = data.iloc[891:]

* You'll use `scikit-learn`, which requires your data as arrays, not DataFrames so transform them:

In [ ]:
x = data_train.values
test = data_test.values
y = survived_train.values

* Now you get to build your decision tree classifier! First create such a model with `max_depth=3` and then fit it your data:

In [ ]:
# Instantiate model and fit to data; clf = classifier
clf = tree.DecisionTreeClassifier(max_depth=3)
clf.fit(x, y)

* Make predictions on your test set, create a new column 'Survived' and store your predictions in it. Save 'PassengerId' and 'Survived' columns of `df_test` to a .csv and submit to Kaggle.

In [ ]:
# Make predictions and store in 'Survived' column of df_test
y_pred = clf.predict(test)
df_test['Survived'] = y_pred

In [ ]:
df_test[['PassengerId', 'Survived']].to_csv('predictions/1st_dec_tree.csv', index=False)

* What is the accuracy of your model, as reported by Kaggle?

Accuracy = 77.99

**Recap:**
* You've got your data in a form to build first machine learning model.
* You've built your first machine learning model: a decision tree classifier.

**Up next:** figure out what this `max_depth` argument was, why we chose it and explore `train_test_split`.


## What was this decision tree classifier?

<img src="img/decision_tree_titanic_3.png" width="900">

Note: you can use `graphviz` to generate figures such as this. See the `scikit-learn` documentation [here](http://scikit-learn.org/stable/modules/tree.html) for further details. In building this model, what you're essentially doing is creating a _decision boundary_ in the space of feature variables, for example (image from [here](http://scikit-learn.org/stable/auto_examples/ensemble/plot_voting_decision_regions.html)):

<img src="img/dec_bound.png" width="400">

## Why would you choose max_depth=3 ?

The depth of the tree is known as a hyperparameter, which means a parameter we need to decide before we fit the model to the data. If we choose a larger `max_depth`, we'll get a more complex decision boundary. 

* If our decision boundary is _too complex_ we can overfit to the data, which means that our model will be describing noise as well as signal.

* If our max_depth is too small, we may be underfitting the data, meaning that our model doesn't contain enough of the signal.

**How do we tell whether we're overfitting or underfitting?** Note: this is also referred to as the bias-variance trade-off and we won;t go into details on that here.

One way is to hold out a test set from our training data. We can then fit the model to our training data, make predictions on our test set and see how well our prediction does on the test set. 

As a reminder, x are the features and y are the targets.

* You'll now do this: split your original training data into training and test sets:

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.33, random_state=42, stratify=y)

* Iterate over values of `max_depth` ranging from 1 to 9 and plot the accuracy of the models on training and test sets:

In [ ]:
# Setup arrays to store train and test accuracies
dep = np.arange(1, 9)
train_accuracy = np.empty(len(dep))
test_accuracy = np.empty(len(dep))

# Loop over different values of k
for i, k in enumerate(dep):
    # Setup a k-NN Classifier with k neighbors: knn
    clf = tree.DecisionTreeClassifier(max_depth=k)

    # Fit the classifier to the training data
    clf.fit(x_train, y_train)
    
    #Compute accuracy on the training set
    train_accuracy[i] = clf.score(x_train, y_train)

    #Compute accuracy on the testing set
    test_accuracy[i] = clf.score(x_test, y_test)

# Generate plot
plt.title('clf: Varying depth of tree')
plt.plot(dep, test_accuracy, label = 'Testing Accuracy')
plt.plot(dep, train_accuracy, label = 'Training Accuracy')
plt.legend()
plt.xlabel('Depth of tree')
plt.ylabel('Accuracy')
plt.show()

**Recap:**
* You've got your data in a form to build first machine learning model.
* You've built your first machine learning model: a decision tree classifier.
* You've learnt about `train_test_split` and how it helps us to choose ML model hyperparameters.

**Up next:** Engineer some new features and build some new models! Open the notebook `3-titanic_feature_engineering_ML.ipynb`.
